<a href="https://colab.research.google.com/github/El-Do-RaDo/RNN-from-Scratch/blob/master/RNN_using_numpy_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://camo.githubusercontent.com/d5dd988bfbca4654bfcecefe35a03a07f3356455/68747470733a2f2f69616d747261736b2e6769746875622e696f2f696d672f62617369635f726563757272656e63655f73696e676c65746f6e2e706e67)

Our steps

*   Initialize weights randomly

*   Give the model a char pair (input char & target char. The target char is the char the network should guess, its the next char in our sequence)


*   Forward pass (We calculate the probability for every possible next char according to the state of the model, using the paramters)



*   Measure error (the distance between the previous probability and the target char)


*   We calculate gradients for each of our parameters to see their impact they have on the loss (backpropagation through time)


*   update all parameters in the direction via gradients that help to minimise the loss


*   Repeat! Until our loss is small AF

In [0]:
from google.colab import files

In [0]:
upload = files.upload()

Saving kafka.txt to kafka (3).txt


In [0]:
data = open('kafka (3).txt','r').read()

char = list(set(data))
data_size, vocab_size = len(data), len(char)
print('our Data has {} chars and {} unique chars'.format(data_size, vocab_size))

our Data has 137628 chars and 80 unique chars


In [0]:
char_to_ix = {ch:i for i,ch in enumerate(char)}
ix_to_char = {i:ch for i, ch in enumerate(char)}

print(char_to_ix)
print(ix_to_char)

In [0]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


The model parameters are adjusted during the trainning.


*   Wxh are parameters to connect a vector that contain one input to the hidden layer.


*   Whh are parameters to connect the hidden layer to itself. This is the Key of the Rnn: Recursion is done by injecting the previous values from the output of the hidden state, to itself at the next iteration.


*   Why are parameters to connect the hidden layer to the output


*   bh contains the hidden bias


*   by contains the output bias

In [0]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 
Why = np.random.randn(vocab_size, hidden_size) * 0.01 
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [0]:
def lossFun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}

    hs[-1] = np.copy(hprev)

    loss = 0

    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)                                                                                                             
        ys[t] = np.dot(Why, hs[t]) + by                                                                                                          
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))                                                                                                           
        loss += -np.log(ps[t][targets[t],0])

        dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
        dbh, dby = np.zeros_like(bh), np.zeros_like(by)
        dhnext = np.zeros_like(hs[0])

        for t in reversed(range(len(inputs))):
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1 
            dWhy += np.dot(dy, hs[t].T)
            dby += dy
            dh = np.dot(Why.T, dy) + dhnext                                                                                                                            
            dhraw = (1 - hs[t] * hs[t]) * dh                                                                                                                     
            dbh += dhraw 
            dWxh += np.dot(dhraw, xs[t].T) 
            dWhh += np.dot(dhraw, hs[t-1].T) 
            dhnext = np.dot(Whh.T, dhraw) 
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)                                                                                                               
        return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]



In [0]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)


    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1))
sample(hprev,char_to_ix['a'],200)

----
 8ça7/7V'H%0X@p--v8BfOA)RQ4Py:7w"dxj4NpWTgfti,N3sg)xT'zJ,E8
.ny
KOçno!.yxu5aPx8iHUK9VS.RavIdF4nO(r;ç.BonQ%TenGwdwQ(q"Cg%V3w% 8Q P3:p1%%"tfYnDr63fjo;B4??;bKf)v5Vçxz/çC3MRw:S1fv:
K5ld4%qIU$tCUu;%QON
1uFh 
----


In [0]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [67, 38, 77, 34, 19, 20, 14, 38, 57, 38, 16, 24, 34, 0, 9, 77, 38, 34, 12, 14, 77, 16, 20, 14, 34]
targets [38, 77, 34, 19, 20, 14, 38, 57, 38, 16, 24, 34, 0, 9, 77, 38, 34, 12, 14, 77, 16, 20, 14, 34, 40]


In [0]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)                                                                                                                 
smooth_loss = -np.log(1.0/vocab_size)*seq_length                                                                                                                      
while n<=1000*100:
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    if n % 1000 == 0:
        print ('iter %d, loss: %f' % (n, smooth_loss)) 
        sample(hprev, inputs[0], 200)

    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],[dWxh, dWhh, dWhy, dbh, dby],[mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)   

    p += seq_length
    n += 1 